<a href="https://colab.research.google.com/github/YAMAZAKIAkiraRK/GCAnet/blob/main/DataAugTest_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/GCA/

/content/drive/MyDrive/Colab Notebooks/GCA


In [4]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3

import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn.preprocessing as sp
import cv2
from sklearn.model_selection import train_test_split
import preprocess as pp
import Grad_Cam as gc

In [5]:
#訓練データの読み込み
p_dir = "YU/grating image/202207/GC2022"
c_dir = os.listdir(p_dir)
i = 0
data=[]
label=[]
label_name = []

for d in c_dir:
    label_name.append(d)
    d = os.path.join(p_dir, d)
    pp.append_data(d, data, label, i)
    i += 1
label = np.array(label).reshape(-1,1)
#ラベルをone_hot_encoding
enc = sp.OneHotEncoder(categories="auto", sparse_output=False, dtype=np.float32)
label_enc = enc.fit_transform(label)


#テストデータの読み込み
p_dir = "YU/grating image/202207/test data"
c_dir = os.listdir(p_dir)
i = 0
test_data=[]
test_label=[]
test_label_name = []
for d in c_dir:
    test_label_name.append(d)
    d = os.path.join(p_dir, d)
    pp.append_data(d, test_data, test_label, i)
    i += 1

In [6]:
#前処理-アスペクト比は気にせず299,299に縮小
#訓練データ
shrink_data = np.empty((len(data),299,299,3))
for i in range(len(data)):
    shrink_data[i]=cv2.resize(data[i], dsize=(299, 299))
shrink_data = np.array(shrink_data)

#テストデータ
shrink_test_data = np.empty((len(test_data),299,299,3))
for i in range(len(test_data)):
    shrink_test_data[i]=cv2.resize(test_data[i], dsize=(299, 299))
shrink_test_data = np.array(shrink_test_data)

In [7]:
params = {
    'rotation_range': 15,
    'vertical_flip': True,
    'horizontal_flip': True,
    'height_shift_range': 0.05,
    'width_shift_range': 0.05,
    'channel_shift_range': 127,
    'zoom_range': [0.05, 1.0],
}

In [8]:
def gengenerator(data, label, param):
    train_data, val_data, train_label, val_label = train_test_split(data, label, random_state=1, stratify = label)
    data_gen = keras.preprocessing.image.ImageDataGenerator(param)
    data_gen.fit(data)
    train_generator = data_gen.flow(train_data, train_label, batch_size=32)
    val_generator = data_gen.flow(val_data, val_label)
    return train_generator, val_generator

In [9]:
#パラメータを渡し、正答率を返す関数
def DAcomparison(data, label, test_data, test_label, params):
    models=[]
    histories=[]
    preds=[]
    accuracies = []
    for key, val in params.items():
        param = {key:val}

        train_generator, val_generator  = gengenerator(data, label, param)

        #GoogLeNetを読み込み、出力層の数を6に変更
        base_model = InceptionV3(weights="imagenet", input_shape=(299, 299,3) ,include_top=False)
        x = base_model.output
        x = keras.layers.GlobalAveragePooling2D()(x)
        predictions = Dense(6, activation="softmax")(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        #モデルのコンパイル
        for layer in base_model.layers:
            layer.trainable = False
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

        train_data, val_data, train_label, val_label = train_test_split(data, label_enc, random_state=1, stratify = label)
        data_gen = keras.preprocessing.image.ImageDataGenerator(param)




        history = model.fit_generator(data_gen.flow(train_data, train_label, batch_size=4),
                                      steps_per_epoch = len(train_data)/4,
                                      validation_data = data_gen.flow(val_data, val_label),
                                      epochs=120, verbose=1)
        models.append(model)
        histories.append(history)
        prediction = np.argmax(model.predict(test_data),axis=1)
        preds.append(prediction)
        accuracy = sum(test_label==prediction) / len(test_label)
        accuracies.append(accuracy)

    return models, histories, preds, accuracies

In [ ]:
models, histories, predictions, accuracies = DAcomparison(
    data=shrink_data, label=label, test_data=shrink_test_data, test_label=test_label, params=params
)

87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/120


<ipython-input-9-414dda0408a1>:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(data_gen.flow(train_data, train_label, batch_size=4),
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


10/10 [==============================] - 25s 2s/step - loss: 5.1723 - accuracy: 0.1220 - val_loss: 4.2097 - val_accuracy: 0.3571
Epoch 2/120
10/10 [==============================] - 13s 1s/step - loss: 3.0081 - accuracy: 0.2439 - val_loss: 2.6869 - val_accuracy: 0.2143
Epoch 3/120
10/10 [==============================] - 14s 1s/step - loss: 2.0484 - accuracy: 0.0976 - val_loss: 2.3398 - val_accuracy: 0.3571
Epoch 4/120
10/10 [==============================] - 16s 2s/step - loss: 1.8516 - accuracy: 0.2927 - val_loss: 1.9844 - val_accuracy: 0.5000
Epoch 5/120
10/10 [==============================] - 16s 2s/step - loss: 1.6207 - accuracy: 0.2927 - val_loss: 1.7682 - val_accuracy: 0.4286
Epoch 6/120
10/10 [==============================] - 14s 1s/step - loss: 1.4253 - accuracy: 0.3902 - val_loss: 1.6321 - val_accuracy: 0.5714
Epoch 7/120
10/10 [==============================] - 14s 1s/step - loss: 1.3438 - accuracy: 0.5122 - val_loss: 1.5458 - val_accuracy: 0.5714
Epoch 8/120
10/10 [======

In [ ]:
model = models[6]
model.summary()

IndexError: ignored

In [ ]:
model.layers[-1].activation = None
img_path = "shrink/shrink_testdata/shrink_testdata16.jpg"
img_array = gc.get_img_array(img_path)
last_conv_layer_name = "conv2d_657"
heatmap = gc.make_gradcam_heatmap(img_array, model, last_conv_layer_name)
gc.save_and_display_gradcam(img_path, heatmap, cam_path="cam6.jpg")

In [ ]:
def NonDAcomparison(data, label, test_data, test_label):

  #GoogLeNetを読み込み、出力層の数を6に変更
  base_model = InceptionV3(weights="imagenet", input_shape=(299, 299,3) ,include_top=False)
  x = base_model.output
  x = keras.layers.GlobalAveragePooling2D()(x)
  predictions = Dense(6, activation="softmax")(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  #モデルのコンパイル
  for layer in base_model.layers:
      layer.trainable = False
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

  train_data, val_data, train_label, val_label = train_test_split(data, label, random_state=1, stratify = label)

  history = model.fit(train_data, train_label, batch_size=4,
                      steps_per_epoch = len(train_data)/4,
                      validation_data = (val_data, val_label),
                      epochs=300, verbose=1)
  prediction = np.argmax(model.predict(test_data),axis=1)
  accuracy = sum(test_label==prediction) / len(test_label)

  return model, history, prediction, accuracy

In [ ]:
NDAmodel, NDAhistory, NDAprediction, NDAaccuracy = NonDAcomparison(
    data=shrink_data, label=label_enc, test_data=shrink_test_data, test_label=test_label
    )

Epoch 1/300
10/10 [==============================] - 7s 317ms/step - loss: 5.8078 - accuracy: 0.2927 - val_loss: 4.2452 - val_accuracy: 0.1429
Epoch 2/300
10/10 [==============================] - 0s 49ms/step - loss: 4.4691 - accuracy: 0.1951 - val_loss: 3.5492 - val_accuracy: 0.1429
Epoch 3/300
10/10 [==============================] - 0s 43ms/step - loss: 3.4944 - accuracy: 0.3415 - val_loss: 2.9523 - val_accuracy: 0.4286
Epoch 4/300
10/10 [==============================] - 0s 48ms/step - loss: 2.7124 - accuracy: 0.4634 - val_loss: 2.1702 - val_accuracy: 0.5000
Epoch 5/300
10/10 [==============================] - 0s 50ms/step - loss: 1.8561 - accuracy: 0.4878 - val_loss: 1.4992 - val_accuracy: 0.5000
Epoch 6/300
10/10 [==============================] - 1s 51ms/step - loss: 1.2399 - accuracy: 0.4634 - val_loss: 1.3502 - val_accuracy: 0.5000
Epoch 7/300
10/10 [==============================] - 0s 46ms/step - loss: 1.1498 - accuracy: 0.5610 - val_loss: 1.2028 - val_accuracy: 0.5000
Epoch

In [1]:
model = NDAmodel
model.summary()

NameError: ignored

In [ ]:
model.layers[-1].activation = None
img_path = "shrink/shrink_testdata/shrink_testdata16.jpg"
img_array = gc.get_img_array(img_path)
last_conv_layer_name = "conv2d_939"
heatmap = gc.make_gradcam_heatmap(img_array, model, last_conv_layer_name)
gc.save_and_display_gradcam(img_path, heatmap, cam_path="camNDA.jpg")

In [ ]:
accuracies = []

In [ ]:
for i in range(6):
  model = models[i]
  model.layers[-1].activation = True
  predict = model.predict(shrink_test_data)
  prediction = np.argmax(predict,axis=1)
  accuracy = sum(tl == prediction) / 41
  accuracies.append(accuracy)

2/2 [==============================] - 0s 105ms/step


In [ ]:
accuracies

[0.024390243902439025,
 0.0975609756097561,
 0.0975609756097561,
 0.07317073170731707,
 0.07317073170731707,
 0.04878048780487805]

In [ ]:
model = NDAmodel
predict = model.predict(shrink_test_data)
prediction = np.argmax(predict,axis=1)
accuracy = sum(tl == prediction) / 41
accuracies.append(accuracy)

2/2 [==============================] - 0s 158ms/step


In [ ]:
accuracies

[0.024390243902439025,
 0.0975609756097561,
 0.0975609756097561,
 0.07317073170731707,
 0.07317073170731707,
 0.04878048780487805,
 0.07317073170731707]

In [ ]:
predict = NDAmodel.predict(shrink_test_data)
accuracy = sum(tl == prediction) / 41
accuracy

2/2 [==============================] - 0s 152ms/step


0.07317073170731707

In [ ]:
pred_label_name = []
for i in range(len(l)):
  nm = label_name[l[i]]
  pred_label_name.append(nm)
pred_label_name

In [ ]:
correct_label_name = []
for i in range(len(test_label)):
  nm = test_label_name[test_label[i]]
  correct_label_name.append(nm)
correct_label_name

In [ ]:
import pandas as pd
df_shrink = pd.DataFrame()
df_shrink["Predict_Class"] = pred_label_name
df_shrink["True_Class"] = correct_label_name
print(pd.crosstab(df_shrink["Predict_Class"], df_shrink["True_Class"]))
print("サンプル数：", len(df_shrink["Predict_Class"]))
print("正解数：", sum(df_shrink["Predict_Class"] == df_shrink["True_Class"]))